In [1]:
import os,shutil,pickle,tqdm,sys,random,re,string, datetime,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import tensorflow.keras


import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import shuffle
from math import log, floor

from tensorflow.keras.utils import multi_gpu_model

# from keras import backend as K
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.utils import *
from tensorflow.keras import *
from tensorflow.keras.optimizers import *

import gensim
from gensim.models.word2vec import *
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *

from tensorflow.keras import regularizers
import unicodedata as udata
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import *

from tensorflow.keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
# from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
# from keras_transformer.position import TransformerCoordinateEmbedding
# from keras_transformer.transformer import TransformerACT, TransformerBlock
# from keras_transformer.bert import (
#     BatchGeneratorForBERT, masked_perplexity,
#     MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
# from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention
import gc
from tensorflow.keras import backend as K


Using TensorFlow backend.


In [2]:
root_dir = './data/20news-bydate-v3/'
trainXr = pickle.load(open(root_dir + 'trainXr.pkl','rb'))
trainXg = pickle.load(open(root_dir + 'trainXg.pkl','rb'))
trainY = pickle.load(open(root_dir + 'trainY.pkl','rb'))

trainYf , validYf, testYf = pickle.load(open(root_dir + 'Yf.pkl','rb'))
trainYf = np.expand_dims(trainYf,axis=-1)
validYf = np.expand_dims(validYf,axis=-1)
testYf = np.expand_dims(testYf,axis=-1)
print('train of sent2vec vector:',trainXr.shape,trainXg.shape,trainY.shape, trainYf.shape)

validXr = pickle.load(open(root_dir + 'validXr.pkl','rb'))
validXg = pickle.load(open(root_dir + 'validXg.pkl','rb'))
validY = pickle.load(open(root_dir + 'validY.pkl','rb'))

testXr = pickle.load(open(root_dir + 'testXr.pkl','rb'))
testXg = pickle.load(open(root_dir + 'testXg.pkl','rb'))
testY = pickle.load(open(root_dir + 'testY.pkl','rb'))

print('test of sent2vec vector:',testXr.shape,testXg.shape,testY.shape, testYf.shape)


emb_matrix = pickle.load(open(root_dir + 'embedding_matrix.pkl','rb'))

train of sent2vec vector: (10556, 512, 1024) (10556, 512) (10556, 20) (10556, 512, 1)
test of sent2vec vector: (3524, 512, 1024) (3524, 512) (3524, 20) (3524, 512, 1)


In [3]:
def _shuffle(X, X2 ,X3,X4):
#     X3 = np.take(train_fam_ans,[0],axis=-1) #只train第幾個familiy
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], X2[randomize],X3[randomize],X4[randomize])

In [4]:
trainXr, trainXg, trainY, trainYf = _shuffle(trainXr, trainXg, trainY, trainYf)
print('valid of sent2vec vector:',validXr.shape,validXg.shape,validY.shape, validYf.shape)

valid of sent2vec vector: (3515, 512, 1024) (3515, 512) (3515, 20) (3515, 512, 1)


## Model Hyper-parameters

In [5]:
emb_dim = validXr.shape[-1] #被除數
# num_heads = 48#除數，要整除
max_length = validXg.shape[-1] # max sequence length
cat_num = validY.shape[-1]

vocabulary_size = emb_matrix.shape[0]
# transformer_depth = 1
# transformer_dropout = 0.1
# l2_reg_penalty = 1e-6#1e-4
# dp_rate = 0.01
hidden_dim = 256

# traina = True #改
batch_size = 16 #改

In [6]:
init = tensorflow.keras.initializers.Ones()
gc.collect()

90

In [7]:
# train_ds = tf.data.Dataset.from_tensor_slices(([trainXr,trainXg],[trainYf,trainY])).shuffle(trainXr.shape[0],reshuffle_each_iteration=True).batch(batch_size)
# valid_ds = tf.data.Dataset.from_tensor_slices(([validXr,validXg],[validYf,validY])).batch(batch_size)

In [8]:
# train_ds = tf.data.Dataset.from_tensors((trainXr,trainXg,trainYf,trainY)).shuffle(trainXr.shape[0],reshuffle_each_iteration=True).batch(batch_size)
# valid_ds = tf.data.Dataset.from_tensors((validXr,validXg,validYf,validY)).batch(batch_size)
# gc.collect()

0

# Model

eager execution

In [7]:
class Embedder(Layer):
    '''
    Input Embedding
    '''
    def __init__(self,max_length=max_length,init=init,vocabulary_size=vocabulary_size,emb_matrix=emb_matrix):
        super(Embedder, self).__init__()
        self.mask = Masking(mask_value=0)
        self.downScale = TimeDistributed(Dense(300,use_bias=False,name='roberta_dnn'),name='TD1')
        # ,kernel_initializer=init
        self.gloveEmb = Embedding(vocabulary_size, 300,weights=[emb_matrix],input_length=max_length
                    ,trainable=False,name='glove_emb')
        
        self.add = Add()
    
    
    def call(self,x):
        xr,xg = x
        xr = self.mask(xr)
        xr = self.downScale(xr)
        xg = self.mask(xg)
        xg = self.gloveEmb(xg)
        return self.add([xr,xg])
# tmp_model = Embedder()
# x1 = tf.random.uniform((1, 498, 1024))
# x2 = tf.random.uniform((1, 498))
# out = tmp_model(x1,x2)
# out.shape=> 1,498,300

In [8]:
class Encoder(Layer):
    def __init__(self,hidden_dim=hidden_dim):
        super(Encoder, self).__init__()
        
        self.emb = Embedder()
        self.rnn = GRU(hidden_dim,return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn = BatchNormalization(name='bn')
        self.att = Attention(name='selfatt') # multi head attention(?) VIZ
    def call(self,x):

        x = self.emb(x)
        x = self.rnn(x)
        x = self.bn(x)
        return self.att([x,x])
# x = self.emb(xr,xg)         xr,xg = x
# tmp_model = Encoder()
# x1 = tf.random.uniform((1, 498, 1024))
# x2 = tf.random.uniform((1, 498))
# out = tmp_model(x1,x2)
# out.shape #=> TensorShape([1, 498, 256])

In [9]:
class Filter(Layer):
    def __init__(self):
        super(Filter,self).__init__()
        self.enc = Encoder()
        self.dnn = TimeDistributed(Dense(1,activation='sigmoid',
                             name='roberta_dnn'),name='TD2')
    def transform(self,x):
        return tf.math.round(x)

    
    def call(self,x):

        x = self.enc(x)
        x3 = self.dnn(x)
        return x3,self.transform(x3)
 #x = self.enc(xr,xg)         xr, xg = x
# tmp_model = Filter()
# x1 = tf.random.uniform((1, 498, 1024))
# x2 = tf.random.uniform((1, 498))
# out = tmp_model(x1,x2)
# out.shape #TensorShape([1, 498, 1])

In [10]:
class Classifier(Model):
    '''
    Final model
    '''
    def __init__(self,hidden_dim=int(hidden_dim/2),cat_num=cat_num,do=0.1):
        super(Classifier,self).__init__()
        self.filter = Filter()
        self.mul =  Multiply()
        self.clf = Bidirectional(LSTM(hidden_dim,dropout=do,recurrent_dropout=do))
        self.out = Dense(cat_num,activation='softmax')
        self.emb = Embedder()
    def call(self,x):

        x3,x1 = self.filter(x)
        x2 = self.filter.enc.emb(x)
        x = self.mul([x1,x2])
        x = self.clf(x)
        y = x3, self.out(x)
        return y #important binary , cat number
# return x3, self.out(x)         xr, xg = x
#         x3,x1 = self.filter(xr,xg)
#         x2 = self.filter.enc.emb(xr,xg)
# tmp_model = Classifier()
# x1 = tf.random.uniform((1, 498, 1024))
# x2 = tf.random.uniform((1, 498))
# out1, out2 = tmp_model(x1,x2)
# out1.shape, out2.shape #(TensorShape([1, 498, 1]), TensorShape([1, 20])

In [12]:
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
model = Classifier()
tf.keras.backend.set_floatx('float32')
gc.collect()

0

In [13]:
loss_object1 = tf.keras.losses.MeanSquaredError()
loss_object2 = tf.keras.losses.CategoricalCrossentropy()
def l2_loss(real,pred):
    '''
    需要創建一個長度為tok length的real來給filter計算mask
    1e-8 ... 0 ...
    '''
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object1(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_cat_acc = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_cat_acc = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [14]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
learning_rate = CustomSchedule(hidden_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [15]:
alpha = 0.1 # filter's loss weight
beta = 0.9 # classifier loss weight

@tf.function
def train_step(xr,xg,yi,yc): #標準答案
    '''
    xr: roberta embedding
    xg: glove encoding
    yi: for filter output pseudo ans
    yc: category ans
    '''
    with tf.GradientTape() as tape:
        pred_imp , pred_cat = model(xr,xg) # 預測答案
        loss = alpha*l2_loss(yi,pred_imp)+beta*loss_object2(yc,pred_cat)
    gradients =  tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_cat_acc(yc, pred_cat)
    
@tf.function
def test_step(xr,xg,yi,yc):
    pred_imp, pred_cat = model(xr,xg)
    loss = alpha*l2_loss(yi,pred_imp)+beta*loss_object2(yc,pred_cat)
    
    test_loss(loss)
    test_cat_acc(yc, pred_cat)

In [ ]:
K.clear_session()
gc.collect()
EPOCHS = 5
for epoch in range(EPOCHS):
    gc.collect()
    for xr, xg, yf, y in train_ds:
        train_step(xr, xg, yf, y)
    for xr_v, xg_v, yf_v, y_v in valid_ds:
        test_step(xr_v, xg_v, yf_v, y_v)
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_cat_acc.result()*100,
                        test_loss.result(),
                        test_cat_acc.result()*100))
    train_loss.reset_states()
    train_cat_acc.reset_states()
    test_loss.reset_states()
    test_cat_acc.reset_states()    
    


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 2.7281951904296875, Accuracy: 5.361879825592041, Test Loss: 2.7002387046813965, Test Accuracy: 5.348506450653076
Epoch 2, Loss: 2.6999619007110596, Accuracy: 4.907161712646484, Test Loss: 2.699943780899048, Test Accuracy: 5.348506450653076


In [27]:
model.call(xr,xg)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



(<tf.Tensor: id=2924, shape=(128, 512, 1), dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        ...,
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Tensor: id=44234, shape=(128, 20), dtype=float32, numpy=
 array([[0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
        [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
        [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
        ...,
        [0.05, 0.05, 0.05, ..., 0.

In [19]:
train_step(xr,xg,yf,y)

TypeError: 'NoneType' object is not callable

graph base

In [67]:
roberta1 = Input(shape=(max_length,1024),name='roberta')
roberta = Masking(mask_value=0)(roberta1)
roberta_emb = TimeDistributed(Dense(300,use_bias=False,kernel_initializer=init,name='roberta_dnn'),name='TD1')(roberta)
#shape=(max_length,emb_dim),,batch_shape=(batch_size,max_length,emb_dim)
glove_ids1 = Input(shape=(max_length,), dtype='int32', name='glove_ids') # 輸入的api funvtion name ID
glove_ids = Masking(mask_value=0)(glove_ids1)
#shape=(max_length,),batch_shape=(batch_size,max_length)
glove_emb = Embedding(vocabulary_size, 300,weights=[emb_matrix],input_length=max_length
                    ,trainable=True,name='glove_emb')(glove_ids) #改

final_embeddings = Add()([roberta_emb,glove_emb])

In [68]:
att_rnn = GRU(256,return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(final_embeddings)
att_rnn = BatchNormalization(name='bn')(att_rnn)
self_att = Attention(name='selfatt')([att_rnn,att_rnn])
# self_att = SeqSelfAttention(attention_activation='hard_sigmoid',name='self_attention')(att_rnn) 
#kernel_initializer=keras.initializers.lecun_normal(), #sigmoid af

In [69]:
f1 = (lambda: tf.constant(0))
f2 = (lambda: tf.constant(1))

# @tf.function
def filter_transform(x):
    return tf.case([(tf.less(x, 0.5), f1)], default=f2)

pick = TimeDistributed(Dense(1,activation='hard_sigmoid',kernel_initializer=init,
                             name='roberta_dnn'),name='TD2')(self_att)
# r = tf.keras.backend.less(pick,0.5)
# r = Lambda(filter_transform)(pick)
# r = tf.keras.backend.switch(tf.keras.backend.less(pick,0.5), f1, f2)
# r = filter_transform(pick)

In [70]:
model = Model(inputs=[roberta1,glove_ids1], outputs=[pick]) #out
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
roberta (InputLayer)            [(None, 498, 1024)]  0                                            
__________________________________________________________________________________________________
glove_ids (InputLayer)          [(None, 498)]        0                                            
__________________________________________________________________________________________________
masking_10 (Masking)            (None, 498, 1024)    0           roberta[0][0]                    
__________________________________________________________________________________________________
masking_11 (Masking)            (None, 498)          0           glove_ids[0][0]                  
____________________________________________________________________________________________

In [78]:
def sum_loss(y_true, y_pred):
    return tf.math.reduce_mean(y_pred)

In [53]:
optimizer = keras.optimizers.Nadam(lr=learning_rate, clipvalue=1.) #改

NameError: name 'keras' is not defined

In [43]:
trainY0 = np.zeros((10553,498,1))
trainY0.shape

(10553, 498, 1)

In [79]:
model.compile(
            'Adam',
            loss=[sum_loss]
            )#{'word_predictions': masked_perplexity})

ValueError: weights can not be broadcast to values. values.rank=0. weights.rank=2. values.shape=(). weights.shape=(None, 498).

In [51]:
model.fit([trainXr,trainXg],[trainY0])

Train on 10553 samples
   32/10553 [..............................] - ETA: 2:15:06

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  assertion failed: [weights can not be broadcast to values.] [weights.shape=] [loss/TD2_loss/Cast:0] [32 498] [values.shape=] [loss/TD2_loss/Sum:0] [498 1] [is_scalar=] [0]
	 [[{{node loss/TD2_loss/weighted_loss/broadcast_weights/assert_broadcastable/AssertGuard/else/_13/Assert}}]]
  (1) Invalid argument:  assertion failed: [weights can not be broadcast to values.] [weights.shape=] [loss/TD2_loss/Cast:0] [32 498] [values.shape=] [loss/TD2_loss/Sum:0] [498 1] [is_scalar=] [0]
	 [[{{node loss/TD2_loss/weighted_loss/broadcast_weights/assert_broadcastable/AssertGuard/else/_13/Assert}}]]
	 [[Reshape_27/_80]]
0 successful operations.
0 derived errors ignored. [Op:__inference_distributed_function_17674]

Function call stack:
distributed_function -> distributed_function


In [11]:
loss_object1 = tf.keras.losses.MeanSquaredError()
loss_object2 = tf.keras.losses.CategoricalCrossentropy()
def l2_loss(real,pred):
    '''
    需要創建一個長度為tok length的real來給filter計算mask
    1e-8 ... 0 ...
    '''
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object1(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_cat_acc = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_cat_acc = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [16]:
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
#     loss_object1 = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
#     loss_object2 = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
#     def l2_loss(real,pred):
#         '''
#         需要創建一個長度為tok length的real來給filter計算mask
#         1e-8 ... 0 ...
#         '''
#         mask = tf.math.logical_not(tf.math.equal(real, 0))
#         loss_ = loss_object1(real, pred)

#         mask = tf.cast(mask, dtype=loss_.dtype)
#         loss_ *= mask

#         return tf.reduce_mean(loss_)
model = Classifier()
model = multi_gpu_model(model, gpus=2)
model.compile(loss=['mean_squared_error','categorical_crossentropy'],metrics=['mae','accuracy'],loss_weights=[0.1,0.9]
              ,optimizer='Nadam')

Instructions for updating:
Use `tf.distribute.MirroredStrategy` instead.


ValueError: in converted code:

    <ipython-input-10-f4bd09cd5fcf>:14 call  *
        x3,x1 = self.filter(x)
    /home/leoqaz12/.local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:842 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    <ipython-input-9-db14f5f91241>:13 call  *
        x = self.enc(x)
    /home/leoqaz12/.local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:842 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    <ipython-input-8-bd4707e358ea>:12 call  *
        x = self.emb(x)
    /home/leoqaz12/.local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:842 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    <ipython-input-7-d31b59327bce>:17 call  *
        xr,xg = x

    ValueError: not enough values to unpack (expected 2, got 0)


In [12]:
# gc.collect()
# model.fit(train_ds,epochs=10,validation_data=valid_ds,max_queue_size=32, workers=24, use_multiprocessing=True)

NameError: name 'train_ds' is not defined

In [15]:
gc.collect()
model.fit([trainXr,trainXg],[trainYf,trainY],epochs=10,validation_data=([validXr,validXg],[validYf,validY]),
          batch_size=32,max_queue_size=64, workers=24, use_multiprocessing=True)

Train on 10556 samples, validate on 3515 samples
Epoch 1/10
10556/10556 [==============================] - 1489s 141ms/sample - loss: 2.7092 - output_1_loss: 0.1691 - output_2_loss: 2.9915 - output_1_mae: 0.3656 - output_1_accuracy: 0.7909 - output_2_mae: 0.0950 - output_2_accuracy: 0.0537 - val_loss: 2.7037 - val_output_1_loss: 0.1414 - val_output_2_loss: 2.9888 - val_output_1_mae: 0.3344 - val_output_1_accuracy: 0.7904 - val_output_2_mae: 0.0949 - val_output_2_accuracy: 0.0532
Epoch 2/10
10556/10556 [==============================] - 1469s 139ms/sample - loss: 2.7024 - output_1_loss: 0.1197 - output_2_loss: 2.9894 - output_1_mae: 0.3073 - output_1_accuracy: 0.7909 - output_2_mae: 0.0949 - output_2_accuracy: 0.0547 - val_loss: 2.6998 - val_output_1_loss: 0.1001 - val_output_2_loss: 2.9891 - val_output_1_mae: 0.2813 - val_output_1_accuracy: 0.7904 - val_output_2_mae: 0.0949 - val_output_2_accuracy: 0.0555
Epoch 3/10
10556/10556 [==============================] - 1466s 139ms/sample - lo